In this lab, we're going to focus on a single classification problem: predicting penguins' sex.

## Training a basic model

### Question 1

First, let's choose two numeric features to use for this task. Produce a scatterplot that would be helpful to do this, and choose two numeric features which look like they would be good predictors.

In [52]:
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

penguins_df = sns.load_dataset('penguins') 

In [53]:
penguins_df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


In [ ]:
...

In [54]:
two_chosen_numeric_features = ['bill_length_mm', 'body_mass_g']

### Question 2

Now, let's predict sex using a Logistic Regression model. Fill in the code below to train and evaluate this model.

In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

target_column = 'sex'
model_data = penguins_df[two_chosen_numeric_features + [target_column]]

In [93]:


#  Recode the target column to be 0/1, where 1 indicates that
#  the penguin is female
#  we could use OneHotEncoder, but let's use pandas instead:
model_data['sex'] = model_data['sex'].apply(lambda x: np.nan if pd.isna(x) else int(x == 'Female'))
# If we used `model_data['sex'] = model_data['sex'] == 'Female'`, we'd replace the `na` values with False

#  There are going to be missing features and missing targets,
#  which should be dealt with differently. 

#  Drop the data which has a missing target variable
model_data = model_data[model_data['sex'].notna()]

#  We don't want to drop observations with missing features
#  Instead, we're going to impute values and add an indicator that the original 
#  data was missing

#  Create variables for each feature which indicate whether any of the features are missing
#  Rename them so the columns end in "_is_na"
na_indicators = model_data[two_chosen_numeric_features].isna().rename(lambda x: x + '_is_na', axis=1)
model_data = pd.concat((model_data, na_indicators), axis=1)

#  Impute the mean for missing data
for col in two_chosen_numeric_features:
    model_data[col] = model_data[col].fillna(model_data[col].mean())

/var/folders/49/0rxntbsn5hs0bhv6nxbm70700000gn/T/ipykernel_4946/2013293562.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['sex'] = model_data['sex'].apply(lambda x: np.nan if pd.isna(x) else int(x == 'Female'))


In [96]:
#  Split the data test/train
X_train, X_test, y_train, y_test = train_test_split(
    model_data[two_chosen_numeric_features], 
    model_data[target_column], 
    test_size=0.2, random_state=42
)


# By default, sklearn's LogisticRegression uses L2 regularization, turn this off by passing penalty=None
model = LogisticRegression(penalty=None)
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

y_test


30     1.0
317    1.0
79     0.0
201    1.0
63     0.0
      ... 
288    1.0
4      1.0
83     0.0
319    0.0
66     1.0
Name: sex, Length: 67, dtype: float64

## Evaluating the model

### Question 3

Now we're going to evaluate the performance of this model. Produce the confusion matrices for the test and train predictions, where each cell shows the proportion of the observations in that category. Does the model perform better on the test set or the train set?

In [ ]:
from sklearn.metrics import confusion_matrix
print("Train confusion matrix")

train_cm = confusion_matrix(y_train, y_train_hat)
#  confusion_matrix() returns counts, turn into proportions
train_cm = ...
train_cm

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay(train_cm).plot()

In [ ]:
print("Test confusion matrix")
test_cm = confusion_matrix(y_test, y_test_hat)
test_cm = ...
test_cm

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay(test_cm).plot()

### Question 4

Calculate the following metrics: accuracy, false positive and false negative rates, precision and recall, and F1 score (some of these are implemented in `sklearn.metrics`, some you'll have to write yourself based on the confusion matrix).

We're going to be looking at these metrics again, so write a function which takes the true and labels as inputs and returns all of these values in a dictionary.

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def evaluate_predictions(y_true, y_predicted):
    ...

    return {
        ...
        ...
        ...
        ...
        ...
        ...
    }

evaluate_predictions(y_test, y_test_hat)


In [ ]:
evaluate_predictions(y_train, y_train_hat)


### Question 5

Produce the ROC plot for this classifier. Does it perform better or worse than chance?

In [ ]:
from sklearn.metrics import RocCurveDisplay


#  Get predicted probabilities for both classes for the test set from the model
y_proba_test = ...

# Predicted probability that the observation is female (a single column)
y_proba_female  = ...

RocCurveDisplay.from_predictions(
    y_test,
    y_proba_female,
    plot_chance_level=True,
)


### Question 6

Produce a calibration table and plot from the model's predictions. In which bucket is the model worst calibrated?

In [ ]:
calibration_df = pd.DataFrame({
    'true_label': y_test,
    'predicted_probability': y_proba_female,
})

# Use pandas cut method to produce 10 bins
calibration_df['predicted_probability_bucket'] = pd.cut(calibration_df['predicted_probability'], bins= ...

# The buckets are represented by left and right bounds, calculate the center
calibration_df['predicted_probability_bucket_center'] = calibration_df['predicted_probability_bucket'].apply(lambda x: x.left + (x.right - x.left) / 2).astype(float)

# Calculate the proportion of positive labels in each bucket
# as a DataFrame with two columns, so that it can be plotted by seaborn
prop_positive_by_bucket = ...
prop_positive_by_bucket

In [ ]:
p = sns.lineplot(
    data=prop_positive_by_bucket,
    x='predicted_probability_bucket_center',
    y='true_label',
)
p.set_title('Calibration plot')

In [ ]:
prop_positive_by_bucket['calibration_error'] = ...
prop_positive_by_bucket.sort_values('calibration_error')

# Question 7

A "no-skill" classifier is often used as a baseline for model performance. It predicts that every observation is in the most common class, with probability equal to the proporition of observations in that class. Produce no-skill predictions and evaluate them using your function from above and the ROC curve.

What is the accuracy for the no-skill classifier?

In [ ]:
prop_observations_female = ...

# Predicted probability of label=1 for each observation
y_proba_test_no_skill = ...

# Prediction for each observation
y_hat_test_no_skill = ...

evaluate_predictions(y_test, y_hat_test_no_skill)




In [ ]:
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(
    y_test,
    y_proba_test_no_skill,
    plot_chance_level=True,
)


In [ ]:
no_skill_accuracy = ...
no_skill_accuracy

# Constructing and selecting features



### Question 8

We've only used two numeric features so far in our model. Let's prepare the dataset so that we can use all of the features.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

target_column = 'sex'

model_data = penguins_df[~penguins_df[target_column].isna()].reset_index()
target = model_data[target_column] == 'Female'

#  Using `.dtypes`, select the numeric and categorical feature columns
categorical_feature_columns = ...
numeric_feature_columns = ...

# Remove the target colum from the categorical and numeric feature columns
categorical_feature_columns = [c for c in categorical_feature_columns if c != target_column] # SOLUTION
numeric_feature_columns = [c for c in numeric_feature_columns if c != target_column] # SOLUTION
categorical_feature_columns,  numeric_feature_columns, target_column

In [ ]:
categorical_model_data = model_data[categorical_feature_columns]
categorical_na_indicators =  categorical_model_data.isna().rename(lambda x: f'{x}_isna', axis=1)
#  Fill in the missing values using the most common value rather than the mean
#  (which wouldn't make sense for cateogorical data)
categorical_model_data = categorical_model_data.fillna(
    ...
)


# Use a one-hot encoder to encode all the categorical columns
enc = OneHotEncoder(drop='first', sparse_output=False) 
categorical_model_data = ...
#  Assign the column names back to the output for readability
categorical_model_data = pd.DataFrame(categorical_model_data, columns=enc.get_feature_names_out())

categorical_model_data = pd.concat((categorical_model_data, categorical_na_indicators), axis=1)


In [ ]:
numeric_model_data = model_data[numeric_feature_columns]

numeric_na_indicators =  numeric_model_data.isna().rename(lambda x: f'{x}_isna', axis=1)
numeric_model_data = numeric_model_data.fillna(
    numeric_model_data.mean()
) 
numeric_model_data = pd.concat((numeric_model_data, numeric_na_indicators), axis=1)


# Create the model feature data by concatenating together the numeric and encoded categorical columns
model_data = pd.concat((numeric_model_data, categorical_model_data), axis=1)

X_train, X_test, y_train, y_test = train_test_split(model_data, target, test_size=0.2, random_state=42)


### Question 9

Fit a logistic regression model without regularization and an L1 (LASSO) regularized model to the training data. 
- Which features are selected, and which are dropped?
- Compare the performance of these two models

In [ ]:
logistic_model = LogisticRegression(penalty=None)
logistic_model.fit(X_train, y_train)
feature_names = ...
logistic_model_coefs = pd.DataFrame({
    'name':feature_names, 
    'coef_logistic':logistic_model.coef_.reshape(-1), 
})
logistic_model_coefs

In [ ]:
regularized_logistic_model = LogisticRegression(penalty='l1', C=0.2, solver='liblinear')
regularized_logistic_model.fit(X_train, y_train)
feature_names = ...
regularized_logistic_model_coefs = pd.DataFrame({
    'name':feature_names, 
    'coef_regularized':regularized_logistic_model.coef_.reshape(-1), 
})
regularized_logistic_model_coefs

In [ ]:
coefs_compared = logistic_model_coefs.merge(regularized_logistic_model_coefs)
coefs_compared

In [ ]:
evaluate_predictions(y_test, logistic_model.predict(X_test))

In [ ]:
evaluate_predictions(y_test, regularized_logistic_model.predict(X_test))

## Cross-Validation

### Question 10


Using sklearn's K-fold cross validation, evaluate the preformance of the logistic regression model on the whole dataset. 


In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, random_state=42, shuffle=True)

predictions = []
for n_fold, (train_index, test_index) in enumerate(kf.split(model_data)):
    # Use the train index and test index to extract the right
    #  data for the fold from `model_data` and `target`
    X_train = ...
    y_train = ...
    X_test = ...
    y_test = ...

    model = LogisticRegression(penalty=None, max_iter=10000)
    model.fit(X_train, y_train)
    predictions.append(pd.DataFrame({
        'n_fold': n_fold,
        'y_true_test': y_test,
        'y_pred_test': model.predict(X_test)
    }))
    


In [ ]:
predictions_df = pd.concat(predictions)
evaluate_predictions(
    predictions_df['y_true_test'],
    predictions_df['y_pred_test'],
)

## Additional Questions

### Question 

Calibrate the model's predictions using a second regression model.

### Question 

Produce predictions which are *worse* than those of the no-skill classifier.

### Question 

Use cross validation to choose the best regularization strength and L1 ratio for an elasticnet-penalized Logistic Regression model.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)